In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib_fontja
import joblib

In [19]:
# Excelデータの読み込み
file_path_4 = '/Users/ryoma/BLP/yosoku/【4月予測用】出荷リソース記入用紙.xlsx'  # Excelファイルのパスを指定
sheet_name = '実績昼'  # 読み込むシート名を指定

data_4 = pd.read_excel(file_path_4, sheet_name=sheet_name)
data_4.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,作業ライン,項目,2024-09-01 00:00:00,2024-09-02 00:00:00,2024-09-03 00:00:00,2024-09-04 00:00:00,2024-09-05 00:00:00,2024-09-06 00:00:00,2024-09-07 00:00:00,2024-09-08 00:00:00,...,2024-09-24 00:00:00,2024-09-25 00:00:00,2024-09-26 00:00:00,2024-09-27 00:00:00,2024-09-28 00:00:00,2024-09-29 00:00:00,2024-09-30 00:00:00,1900-01-31 00:00:00,2024-10-01,合計
2,NaN,日付,1,2,3,4,5,6,7,8,...,24,25,26,27,28,29,30,31,NaT,NaN
3,A-PaLS,件数,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,NaN,作業時間(分),NaN,NaN,NaN,4680,1872,NaN,2340,7371,...,7371,7371,NaN,NaN,7371,NaN,7371,NaN,NaT,NaN


In [20]:
# 行列を逆にする
data_4 = data_4.T
# 1行目を列名にする
data_4.columns = data_4.iloc[1]
# 0,1行目を削除
data_4 = data_4.drop(data_4.index[[0, 1]])
data_4.head()

Unnamed: 1,NaN,項目,日付,件数,作業時間(分),作業人数,リーダー人数,合計作業人数,作業ライン,1人あたりの作業時間,1本あたりの平均作業数,1日あたりの作業件数
Unnamed: 2,NaN,2024-09-01 00:00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 3,NaN,2024-09-02 00:00:00,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 4,NaN,2024-09-03 00:00:00,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 5,NaN,2024-09-04 00:00:00,4,NaN,4680,12,3,15,5,312,NaN,NaN
Unnamed: 6,NaN,2024-09-05 00:00:00,5,NaN,1872,4,2,6,2,312,NaN,NaN


In [21]:
data_4 = data_4.drop(data_4.columns[[0, 1, 10,11]], axis=1)
data_4.head()

Unnamed: 1,日付,件数,作業時間(分),作業人数,リーダー人数,合計作業人数,作業ライン,1人あたりの作業時間
Unnamed: 2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 4,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 5,4,NaN,4680,12,3,15,5,312
Unnamed: 6,5,NaN,1872,4,2,6,2,312


In [22]:
# NaNを全て0に変換
data_4 = data_4.fillna(0)
data_4.head()

/var/folders/zn/w212lcms0dxb2bg4lyzl6pxm0000gn/T/ipykernel_16118/4077402185.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_4 = data_4.fillna(0)


Unnamed: 1,日付,件数,作業時間(分),作業人数,リーダー人数,合計作業人数,作業ライン,1人あたりの作業時間
Unnamed: 2,1,0,0,0,0,0,0,0
Unnamed: 3,2,0,0,0,0,0,0,0
Unnamed: 4,3,0,0,0,0,0,0,0
Unnamed: 5,4,0,4680,12,3,15,5,312
Unnamed: 6,5,0,1872,4,2,6,2,312


In [23]:
# Unnamed: 33と34を削除
data_4 = data_4.drop(data_4.index[[31, 32]])
data_4

Unnamed: 1,日付,件数,作業時間(分),作業人数,リーダー人数,合計作業人数,作業ライン,1人あたりの作業時間
Unnamed: 2,1,0,0,0,0,0,0,0
Unnamed: 3,2,0,0,0,0,0,0,0
Unnamed: 4,3,0,0,0,0,0,0,0
Unnamed: 5,4,0,4680,12,3,15,5,312
Unnamed: 6,5,0,1872,4,2,6,2,312
Unnamed: 7,6,0,0,0,0,0,0,0
Unnamed: 8,7,0,2340,3,3,6,2,390
Unnamed: 9,8,0,7371,19,2,21,7,351
Unnamed: 10,9,0,7371,18,3,21,7,351
Unnamed: 11,10,0,7371,19,2,21,7,351


In [26]:
# 「件数」数が0の行を削除
data_4 = data_4[data_4['件数'] != 0]
data_4

Unnamed: 1,日付,件数,作業時間(分),作業人数,リーダー人数,合計作業人数,作業ライン,1人あたりの作業時間


In [ ]:
# データの標準偏差及び平均を取得（正規化解除に使用）
original_mean_4 = data_4.mean()
original_std_4 = data_4.std()
print(original_mean_4)
print(original_std_4)

In [ ]:
data_4 = data_4.apply(lambda x: np.log(x + 1))

In [ ]:
X_test = data_4.drop(columns='件数', errors='ignore')
y_test = data_4['件数']

In [24]:
# 保存したモデルをロード
best_model = joblib.load("warehouse_model.pkl")

print("Model loaded successfully.")


Model loaded successfully.


In [25]:
# 全てのデータに対して
# 予測を行う
y_pred = best_model.predict(X_test)

# 予測結果を元に戻す
y_pred_original = np.exp(y_pred)
y_test_original = np.exp(y_test)

# 各インデックスごとに比較結果を表示
for i in range(len(X_test)):
    print(f'Predicted: {y_pred_original[i]}')
    print(f'Actual: {y_test_original.iloc[i]}')
    print(f'Difference: {y_pred_original[i] - y_test_original.iloc[i]}')
    print('----------')

NameError: name 'X_test' is not defined